In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#Access and recess equation form the Parisian Alfonsines Tables
Hist_table=HTable.read(315,format='dishas')
Hist_table

arg2,Tables [24]
deg,
IntegerAndSexagesimal,void128
11 ;,"(1 ; , 00 ; 27) .. (24 ; , 11 ; 00)"
12 ;,"(1 ; , 00 ; 30) .. (24 ; , 11 ; 00)"
13 ;,"(1 ; , 00 ; 32) .. (24 ; , 13 ; 00)"
14 ;,"(1 ; , 00 ; 35) .. (24 ; , 14 ; 00)"
15 ;,"(1 ; , 00 ; 37) .. (24 ; , 15 ; 00)"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(5):
    for j in range(24):
        Test_recomp[i][1][j][1]=Sexagesimal('0;')
        Delta_recomp[i][1][j][1]=Sexagesimal('0;')


In [16]:
for i in range(5):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
        # initial value for dividend
        A1=Hist_table[i][0]
                 
    for j in range(24):
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
            # second values for dividend
            A2=Hist_table[0][1][j][0]
         
        # computation of the proportion
        with set_precision(tmode=TruncatureMode.ROUND, pmode=1):
            prop=A1*A2/24
            Test_recomp[i][1][j][1]=prop*u.degree
            hist=Hist_table[i][1][j][1]
            Delta_recomp[i][1][j][1]=60*60*(hist-prop)

In [17]:
# residuals visualisation and statistical summaries 1
A=Delta_recomp.to_pandas()
(A.mean().mean(), A.std().std())

(-5.0, 393.0824167174739)

In [18]:
# residuals visualisation and statistical summaries 2
A.astype(float).style.background_gradient(axis=None)

,11 ;,12 ;,13 ;,14 ;,15 ;
1 ;,-60.000000,0.000000,-60.000000,0.000000,-60.000000
2 ;,3600.000000,0.000000,0.000000,0.000000,0.000000
3 ;,-60.000000,0.000000,-60.000000,0.000000,-60.000000
4 ;,0.000000,0.000000,0.000000,0.000000,0.000000
5 ;,-60.000000,0.000000,-60.000000,0.000000,-60.000000
6 ;,3600.000000,0.000000,0.000000,0.000000,0.000000
7 ;,-60.000000,0.000000,-60.000000,0.000000,-60.000000
8 ;,0.000000,0.000000,0.000000,0.000000,0.000000
9 ;,-60.000000,0.000000,-60.000000,0.000000,-60.000000
10 ;,420.000000,0.000000,1800.000000,0.000000,0.000000


In [19]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Prop daily to hourly.csv')
B.to_csv('Prop daily to hourly.zip', index=False, compression=compression_opts)